In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import os
import glob
PROJECT_LOCATION = "/content/drive/MyDrive/playground/medical_image_analysis/Homework2"
os.chdir(PROJECT_LOCATION)

In [ ]:
DATA_LOCATION = "./dataset/"

In [ ]:
import numpy as np
from numpy.random import default_rng
import cv2 as cv
from google.colab.patches import cv2_imshow
from skimage import io
from skimage import color
from PIL import Image
import matplotlib.pyplot as plt
from scipy import ndimage


In [ ]:
def calculateCooccurenceMatrix(grayImg, binNumber, di, dj):
  M = np.zeros((binNumber, binNumber))
  bin_size = 256 // binNumber
  bin_gray_img = grayImg // bin_size
  new_shape = bin_gray_img.shape
  shifted_bin_gray_img = np.zeros(bin_gray_img.shape)
  shifted_bin_gray_img[:,:] = -1



  if di > 0 and dj > 0:
    shifted_bin_gray_img[:-di, :-dj] = bin_gray_img[di: ,dj:]
  elif di > 0 and dj < 0:
    shifted_bin_gray_img[:-di, -dj:] = bin_gray_img[di:, :dj]
  elif di > 0 and dj == 0:
    shifted_bin_gray_img[:-di, :] = bin_gray_img[di:, :]
  elif di < 0 and dj > 0:
    shifted_bin_gray_img[-di:, :-dj] = bin_gray_img[:di, dj:]
  elif di < 0 and dj < 0:
    shifted_bin_gray_img[-di:, -dj:] = bin_gray_img[:di, :dj]
  elif di < 0 and dj == 0:
    shifted_bin_gray_img[-di:, :] = bin_gray_img[:di, :]
  elif di == 0 and dj == 0:
    shifted_bin_gray_img = bin_gray_img
  elif di == 0 and dj > 0:
    shifted_bin_gray_img[:, :-dj] = bin_gray_img[:, dj:]
  elif di == 0 and dj < 0:
    shifted_bin_gray_img[:, -dj:] = bin_gray_img[:, :dj]
  
  

  for i in range(binNumber):
    for j in range(binNumber):
      M[i, j] = np.sum(np.logical_and(bin_gray_img == i, shifted_bin_gray_img == j))

  return M



In [ ]:
def calculateAccumulatedCooccurrenceMatrix(grayImg, binNumber, d):
  accM = np.zeros((binNumber, binNumber))
  for i in [-d, 0, d]:
    for j in [-d, 0, d]:
      if i == 0 and j == 0:
        continue
      accM += calculateCooccurenceMatrix(grayImg, binNumber, i, j)
  return accM



In [ ]:
def inverse_indifference_moment(n_acc_m):
  n = n_acc_m.shape[0]
  i = np.arange(1, n+1)
  j = np.arange(1, n+1)
  i.shape = (n, 1)
  j.shape = (1, n)

  return np.sum(n_acc_m / (1 + np.power(i - j, 2)))

def contrast(n_acc_m):
  n = n_acc_m.shape[0]
  i = np.arange(1, n+1)
  j = np.arange(1, n+1)
  i.shape = (n, 1)
  j.shape = (1, n)

  return np.sum(n_acc_m * np.power(i - j, 2))


def entropy(n_acc_m):
  from numpy import ma
  ma_n_acc_m = ma.log(n_acc_m).filled(0)
  return - np.sum(n_acc_m * ma_n_acc_m)

def correlation(n_acc_m):
  n_i = np.sum(n_acc_m, 1)
  n_j = np.sum(n_acc_m, 0)

  n = n_acc_m.shape[0]
  i = np.arange(1, n+1)
  j = np.arange(1, n+1)

  mu_x = np.sum(n_i * i)
  mu_y = np.sum(n_j * j)
  std_x = np.sqrt(np.sum(n_i * np.power(i - mu_x, 2)))
  std_y = np.sqrt(np.sum(n_j * np.power(j - mu_y, 2)))

  i.shape = (n, 1)
  j.shape = (1, n)

  sum = np.sum((i * j) * n_acc_m)

  return (sum - mu_x * mu_y) / (std_x * std_y)

def calculateCooccurrenceFeatures(accM):
  normalized_acc_m = accM / np.sum(accM)
  ang_nd_moment = np.sum(np.power(normalized_acc_m, 2))
  max_prob = np.max(normalized_acc_m)
  inv_ind_moment = inverse_indifference_moment(normalized_acc_m)
  cont = contrast(normalized_acc_m)
  ent = entropy(normalized_acc_m)
  corr = correlation(normalized_acc_m)
  return [ang_nd_moment, max_prob, inv_ind_moment, cont, ent, corr]

In [ ]:
# get all images and their features
bin_number, d = 8, 10
training_dir = DATA_LOCATION + "training/"
features = []
for img_file in glob.glob(training_dir + "*.jpg"):
  img = cv.imread(img_file, 0)
  acc_m = calculateAccumulatedCooccurrenceMatrix(img, bin_number, d)
  features.append(calculateCooccurrenceFeatures(acc_m))

training_labels = np.loadtxt(DATA_LOCATION + "training_labels.txt")
training_labels = training_labels.tolist()

In [ ]:
test_dir = DATA_LOCATION + "test/"
test_features = []
for img_file in glob.glob(test_dir + "*.jpg"):
  img = cv.imread(img_file, 0)
  acc_m = calculateAccumulatedCooccurrenceMatrix(img, bin_number, d)
  test_features.append(calculateCooccurrenceFeatures(acc_m))

test_labels = np.loadtxt(DATA_LOCATION + "test_labels.txt")
test_labels = test_labels.tolist()

In [ ]:
Cs = [0.1, 1, 5, 10, 50, 100, 250, 500, 1000, 5000]
gammas = [0.1, 1, 5, 10]



In [ ]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

scaler = StandardScaler()
features_std = scaler.fit_transform(features)
features_test_std = scaler.fit_transform(test_features)

In [ ]:
# linear SVM

training_labels = np.array(training_labels)
test_labels = np.array(test_labels)
features_std = np.array(features_std)
features_test_std = np.array(features_test_std)



In [ ]:
for C in list(np.arange(5,10, 0.1)):
  svc = SVC(kernel='linear', class_weight='balanced', C=C)
  model = svc.fit(features_std, training_labels)
  training_pred = np.array(model.predict(features_std))
  test_pred = np.array(model.predict(features_test_std))
  
  print('C : ', C)
  print('Training Overall Accuracy : ', accuracy_score(training_labels, training_pred))
  print('Training Class Accuracy : ', confusion_matrix(training_labels, training_pred, normalize="true").diagonal())
  print('Test Overall Accuracy : ', accuracy_score(test_labels, test_pred))
  print('Test Class Accuracy : ', confusion_matrix(test_labels, test_pred, normalize="true").diagonal())


In [ ]:
for C in [1,5,10]:
  for gamma in list(np.arange(0.1, 1, 0.1)):
    svc = SVC(kernel='rbf', class_weight='balanced', C=C, gamma=gamma)
    model = svc.fit(features_std, training_labels)
    training_pred = np.array(model.predict(features_std))
    test_pred = np.array(model.predict(features_test_std))
    
    print('C : ', C, 'gamma : ', gamma)
    print('Training Overall Accuracy : ', accuracy_score(training_labels, training_pred))
    print('Training Class Accuracy : ', confusion_matrix(training_labels, training_pred, normalize="true").diagonal())
    print('Test Overall Accuracy : ', accuracy_score(test_labels, test_pred))
    print('Test Class Accuracy : ', confusion_matrix(test_labels, test_pred, normalize="true").diagonal())

In [ ]:
# McNemar Linear and RBF comparison

svc = SVC(kernel='linear', class_weight='balanced', C=5.4)
model_linear = svc.fit(features_std, training_labels)
training_pred_linear = np.array(model_linear.predict(features_std))
test_pred_linear = np.array(model_linear.predict(features_test_std))



svc = SVC(kernel='rbf', class_weight='balanced', C=10, gamma=0.1)
model_rbf = svc.fit(features_std, training_labels)
training_pred_rbf = np.array(model_rbf.predict(features_std))
test_pred_rbf = np.array(model_rbf.predict(features_test_std))



In [ ]:
from statsmodels.stats.contingency_tables import mcnemar

def contingency(truth, m1, m2, val=0):
  m = [[0,0],[0,0]]
  tf_m1 = truth == m1
  tf_m2 = truth == m2
  if val != 0:
    tf_m1 = tf_m1[truth == val]
    tf_m2 = tf_m2[truth == val]
  m[0][0] = np.sum(np.logical_and(tf_m1, tf_m2))
  m[0][1] = np.sum(np.logical_and(tf_m1, np.logical_not(tf_m2)))
  m[1][0] = np.sum(np.logical_and(np.logical_not(tf_m1), tf_m2))
  m[1][1] = np.sum(np.logical_and(np.logical_not(tf_m1), np.logical_not(tf_m2)))
  return m

In [ ]:
for i in range(4):
  table = contingency(training_labels, training_pred_linear, training_pred_rbf, i)
  result = mcnemar(table, exact=False, correction=True)
  print(result)

pvalue      0.0093747684594349
statistic   6.75
pvalue      1.0
statistic   0.0
pvalue      0.02334220201289086
statistic   5.142857142857143
pvalue      0.47950012218695337
statistic   0.5


In [ ]:
for i in range(4):
  table = contingency(test_labels, test_pred_linear, test_pred_rbf, i)
  result = mcnemar(table, exact=False, correction=True)
  print(result)

pvalue      1.0
statistic   0.0
pvalue      0.6170750774519739
statistic   0.25
pvalue      0.7236736098317629
statistic   0.125
pvalue      0.24821307898992026
statistic   1.3333333333333333


In [ ]:
# Part III
N = 4
# get all images and their features
bin_number, d = 8, 10

features_patches = []
for img_file in glob.glob(training_dir + "*.jpg"):
  img = cv.imread(img_file, 0)
  x, y = img.shape
  h, w = x // N, y // N
  acc_m_list = []
  for i in range(N):
    for j in range(N):
      patch = img[i*h:(i+1)*h, j*w:(j+1)*w]
      acc_m = calculateAccumulatedCooccurrenceMatrix(patch, bin_number, d)
      acc_m_list.append(calculateCooccurrenceFeatures(acc_m))

  acc_m_np = np.array(acc_m_list)
  features_patches.append(np.mean(acc_m_np, 0))
    
features_patches_np = np.array(features_patches)



In [ ]:
test_features_patches = []

for img_file in glob.glob(test_dir + "*.jpg"):
  img = cv.imread(img_file, 0)
  x, y = img.shape
  h, w = x // N, y // N
  acc_m_list = []
  for i in range(N):
    for j in range(N):
      patch = img[i*h:(i+1)*h, j*w:(j+1)*w]
      acc_m = calculateAccumulatedCooccurrenceMatrix(patch, bin_number, d)
      acc_m_list.append(calculateCooccurrenceFeatures(acc_m))

  acc_m_np = np.array(acc_m_list)
  test_features_patches.append(np.mean(acc_m_np, 0))
    
test_features_patches_np = np.array(test_features_patches)

In [ ]:
Cs = [0.1, 1, 5, 10, 50, 100, 250, 500, 1000, 5000]
gammas = [0.1, 1, 5, 10]

In [ ]:
features_patches_np_std = scaler.fit_transform(features_patches_np)
test_features_patches_np_std = scaler.fit_transform(test_features_patches_np)

In [ ]:
for C in np.arange(5, 10, 0.1):
  svc = SVC(kernel='linear', class_weight='balanced', C=C)
  model = svc.fit(features_patches_np_std, training_labels)
  training_pred = np.array(model.predict(features_patches_np_std))
  test_pred = np.array(model.predict(test_features_patches_np_std))
  
  print('C : ', C)
  print('Training Overall Accuracy : ', accuracy_score(training_labels, training_pred))
  print('Training Class Accuracy : ', confusion_matrix(training_labels, training_pred, normalize="true").diagonal())
  print('Test Overall Accuracy : ', accuracy_score(test_labels, test_pred))
  print('Test Class Accuracy : ', confusion_matrix(test_labels, test_pred, normalize="true").diagonal())

In [ ]:
for C in [50, 100]:
  for gamma in list(np.arange(0.1,1, 0.1)):
    svc = SVC(kernel='rbf', class_weight='balanced', C=C, gamma=gamma)
    model = svc.fit(features_patches_np_std, training_labels)
    training_pred = np.array(model.predict(features_patches_np_std))
    test_pred = np.array(model.predict(test_features_patches_np_std))
    
    print('C : ', C, 'gamma : ', gamma)
    print('Training Overall Accuracy : ', accuracy_score(training_labels, training_pred))
    print('Training Class Accuracy : ', confusion_matrix(training_labels, training_pred, normalize="true").diagonal())
    print('Test Overall Accuracy : ', accuracy_score(test_labels, test_pred))
    print('Test Class Accuracy : ', confusion_matrix(test_labels, test_pred, normalize="true").diagonal())

In [ ]:
# McNemar All image vs Partials for Linear

svc = SVC(kernel='linear', class_weight='balanced', C=5.4)
model_linear = svc.fit(features_std, training_labels)
training_pred_linear = np.array(model_linear.predict(features_std))
test_pred_linear = np.array(model_linear.predict(features_test_std))



svc = SVC(kernel='linear', class_weight='balanced', C=5.2)
model_linear_partial = svc.fit(features_patches_np_std, training_labels)
training_pred_linear_partial = np.array(model_linear_partial.predict(features_patches_np_std))
test_pred_linear_partial = np.array(model_linear_partial.predict(test_features_patches_np_std))

In [ ]:
for i in range(4):
  table = contingency(training_labels, training_pred_linear, training_pred_linear_partial, i)
  result = mcnemar(table, exact=False, correction=True)
  print(result)

pvalue      1.0
statistic   0.0
pvalue      0.4496917979688908
statistic   0.5714285714285714
pvalue      0.6830913983096086
statistic   0.16666666666666666
pvalue      0.47950012218695337
statistic   0.5


In [ ]:
for i in range(4):
  table = contingency(test_labels, test_pred_linear, test_pred_linear_partial, i)
  result = mcnemar(table, exact=False, correction=True)
  print(result)

pvalue      0.1456100953968629
statistic   2.1176470588235294
pvalue      0.18242243945173198
statistic   1.7777777777777777
pvalue      1.0
statistic   0.0
pvalue      0.37109336952269756
statistic   0.8


In [ ]:
# McNemar All image vs Partials for RBF

svc = SVC(kernel='rbf', class_weight='balanced', C=10, gamma=0.1)
model_rbf = svc.fit(features_std, training_labels)
training_pred_rbf = np.array(model_rbf.predict(features_std))
test_pred_rbf = np.array(model_rbf.predict(features_test_std))



svc = SVC(kernel='rbf', class_weight='balanced', C=50, gamma=0.1)
model_rbf_partial = svc.fit(features_patches_np_std, training_labels)
training_pred_rbf_partial = np.array(model_rbf_partial.predict(features_patches_np_std))
test_pred_rbf_partial = np.array(model_rbf_partial.predict(test_features_patches_np_std))

In [ ]:
for i in range(4):
  table = contingency(training_labels, training_pred_rbf, training_pred_rbf_partial, i)
  result = mcnemar(table, exact=False, correction=True)
  print(result)

pvalue      0.2672574931543847
statistic   1.2307692307692308
pvalue      0.13057001811573693
statistic   2.2857142857142856
pvalue      1.0
statistic   0.0
pvalue      1.0
statistic   0.0


In [ ]:
for i in range(4):
  table = contingency(test_labels, test_pred_rbf, test_pred_rbf_partial, i)
  result = mcnemar(table, exact=False, correction=True)
  print(result)

pvalue      0.7892680261342813
statistic   0.07142857142857142
pvalue      1.0
statistic   0.0
pvalue      1.0
statistic   0.0
pvalue      0.0
statistic   inf


/usr/local/lib/python3.7/dist-packages/statsmodels/stats/contingency_tables.py:1336: RuntimeWarning: divide by zero encountered in double_scalars
  statistic = (np.abs(n1 - n2) - corr)**2 / (1. * (n1 + n2))
